# CONFIDENTIAL - DO NOT SHARE

# TFX Developer Workshop

## Introduction

This workshop is designed to introduce TensorFlow Extended (TFX)
and help you learn to create your own machine learning
pipelines.  We'll follow a typical ML development process,
starting by examining the dataset, and end up with a complete
working pipeline.  Along the way we'll explore ways to debug
and update your pipeline, and measure performance.

## Step by Step
We'll gradually create our pipelines by working step by step,
following a typical ML developer process.  Here are the steps:

1. Setup your environment
1. Bring up initial pipeline skeleton
1. Dive into our data
1. Feature engineering
1. Training
1. Analyzing model performance
1. Deployment to production

## Prerequisites

* Linux
* Virtualenv
* Python 2.7
* Git

## Workshop Materials

The code is organized by the steps that we're working on, so
for each step you'll have the code you need and instructions
on what to do with it in the code.py file in the step directory.

## What we're doing

We’re learning how to create an ML pipeline using TFX

* TFX pipelines are appropriate when datasets are large
* TFX pipelines are appropriate when training/serving consistency is important
* TFX pipelines are appropriate when version management for inference is important
* Google uses TFX pipelines for planet-scale ML

We’re following a typical ML development process

* Understanding our data
* Feature engineering
* Training
* Analyze model performance
* Lather, rinse, repeat
* Deploy to production

## Chicago Taxi Dataset

![Taxi](images/taxi.png)

![Chicago taxi](images/chicago.png)

### Goal - Binary classification

Will the customer tip more or less than 20%?

## Step 1: Setup your environment

In a shell:

```bash
virtualenv -p python2.7 tfx-workshop
source tfx-workshop/bin/activate
mkdir tfx; cd tfx

# git clone https://github.com/tensorflow/workshops/TFX-DevSummit-2019.git
# cp -R <citc>/google3/experimental/users/robertcrowe/TFX-DevSummit-2019 .
copy files from GDE shared drive into the 'tfx' directory that you just created.

cd TFX-DevSummit-2019/workshop/setup
./setup_demo.sh
```

## Step 2: Bring up initial pipeline skeleton

### Hello World

In a shell:

```bash
# Open a new terminal window, and in that window ...
source tfx-workshop/bin/activate
airflow webserver

# Open another new terminal window, and in that window ...
source tfx-workshop/bin/activate
airflow scheduler

# Open a browser and go to http://127.0.0.1:8080
# Enable the tfx_example_pipeline_DAG
# Trigger the tfx_example_pipeline_DAG

# In the Airflow web UI, click on tfx_example_pipeline_DAG
# Click on Graph View
# Wait for the Setup component to turn dark green (~1 minutes)
# Use the refresh button on the right or refresh the page
```

* Return to DAGs list page in Airflow
* Trigger tfx_example_pipeline_DAG
* Wait for pipeline to complete
  * All dark green
  * Use refresh on right side or refresh page

![Setup complete](images/step2.png)

## Step 3: Dive into our data

The first task in any data science or ML project is to understand
and clean the data.

* Understand the data types for each feature
* Look for anomalies and missing values
* Understand the distributions for each feature

### Components

![Data Components](images/examplegen1.png)
![Data Components](images/examplegen2.png)

ExampleGen

* Converts input data to tf.Example

StatisticsGen

* Uses TensorFlow Data Validation (TFDV) to create descriptive statistics for dataset and features

SchemaGen

* Uses TensorFlow Data Validation (TFDV) to infer a schema for the dataset

ExampleValidator

* Uses TensorFlow Data Validation (TFDV) to look for anomalies and missing values

### In an editor:

```python
# Add the following code to ~/airflow/dags/tfx_example_pipeline.py
# Add appropriate imports
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen

# Add components to pipeline in create_pipeline()
statistics_gen = StatisticsGen(
  input_data=example_gen.outputs.output)
infer_schema = SchemaGen(stats=statistics_gen.outputs.output)
validate_stats = ExampleValidator(
  stats=statistics_gen.outputs.output,
  schema=infer_schema.outputs.output)

# Add to return
return [example_gen, statistics_gen, infer_schema, validate_stats]
```

* Return to DAGs list page in Airflow
* Trigger tfx_example_pipeline_DAG
* Wait for pipeline to complete
  * All dark green
  * Use refresh on right side or refresh page

![Dive into data](images/step3.png)

### In a shell:

```bash
cd <repo>/workshop/notebooks
jupyter notebook
```

### In a browser:

* Open step3.ipynb
* Follow the notebook

![Dive into data](images/step3notebook.png)

For a more extensive example of using TFDV to explore and validate a
dataset, [see the examples on tensorflow.org](
https://www.tensorflow.org/tfx/data_validation)

## Step 4: Feature engineering

We can increase the predictive quality of our data and/or reduce
dimensionality with feature engineering.

* Feature crosses
* Vocabularies
* Embeddings
* PCA
* Categorical encoding

Write Once - The resulting transforms will be consistent between training
and serving.

### Components

![Transform](images/transform.png)

Transform

* Uses TensorFlow Transform (TFT) to perform data transformations

### In a shell:

```bash
mkdir ~/airflow/plugins/tfx_example
cp <repo>/setup/plugins/tfx_example/__init__.py ~/airflow/plugins/tfx_example
cp <repo>/setup/plugins/tfx_example/transforms.py ~/airflow/plugins/tfx_example
cp <repo>/setup/plugins/tfx_example/features.py ~/airflow/plugins/tfx_example
```

### In an editor:

```python
# Add the following code to ~/airflow/dags/tfx_example_pipeline.py
# Add appropriate imports
from tfx.components import Transform

# Add new modules above PipelineDecorator
# Modules for trainer and transform
plugin_dir = os.path.join(home_dir, 'plugins/tfx_example/')
model = os.path.join(plugin_dir, 'model.py')
transforms = os.path.join(plugin_dir, 'transforms.py')

# Add components to the end of pipeline in create_pipeline()
  transform = Transform(
      input_data=example_gen.outputs.output,
      schema=infer_schema.outputs.output,
      module_file=transforms)

# Add components to return
  return [
      example_gen, statistics_gen, infer_schema, validate_stats, transform
  ]
```

* Return to DAGs list page in Airflow
* Trigger tfx_example_pipeline_DAG
* Wait for pipeline to complete
  * All dark green
  * Use refresh on right side or refresh page

![Feature Engineering](images/step4.png)

## Step 5: Training

Train a TensorFlow model with our nice, clean, transformed data.

* Include the transformations from step 4 so that they are applied
consistently
* Save the results as a SavedModel for production
* Visualize and explore the training process using TensorBoard
* Also save an EvalSavedModel for analysis of model performance

### Components

Trainer

* Orchestrates the training of a TensorFlow model

### In a shell:

```bash
cp <repo>/setup/plugins/tfx_example/model.py ~/airflow/plugins/tfx_example
```

### In an editor:

```python
# Add the following code to ~/airflow/dags/tfx_example_pipeline.py
# Add appropriate imports
from tfx.components import Trainer

# Add components to the end of pipeline in create_pipeline()
  trainer = Trainer(
      module_file=model,
      transformed_examples=transform.outputs.transformed_examples,
      schema=infer_schema.outputs.output,
      transform_output=transform.outputs.transform_output,
      train_steps=10000,
      eval_steps=5000,
      warm_starting=True)

# Add trainer to return
  return [
      example_gen, statistics_gen, infer_schema, validate_stats, transform,
      trainer
  ]
```

* Return to DAGs list page in Airflow
* Trigger tfx_example_pipeline_DAG
* Wait for pipeline to complete
  * All dark green
  * Use refresh on right side or refresh page

![Training a Model](images/step5.png)

### Back on Jupyter

* Open step5.ipynb
* Follow the notebook

![Training a Model](images/step5tboard.png)

## Step 6: Analyzing model performance

Understanding more than just the top level metrics.

* Users experience model performance for their queries only
* Poor performance on slices of data can be hidden by top level
metrics
* Model fairness is important
* Often key subsets of users or data are very important, and may
be small
    * Performance in critical but unusual conditions
    * Performance for key audiences such as influencers

### Components

Evaluator

* Uses TensorFlow Model Analysis to perform deep analysis of the performance of the model that we trained

### In an editor:

```python
# Add the following code to ~/airflow/dags/tfx_example_pipeline.py
# Add appropriate imports
import tensorflow_model_analysis as tfma
from tfx.components import Evaluator

# Add components to the end of pipeline in create_pipeline()
  model_analyzer = Evaluator(
      examples=example_gen.outputs.output,
      model_exports=trainer.outputs.output)

# Add model_analyzer to return
  return [
      example_gen, statistics_gen, infer_schema, validate_stats, transform,
      trainer, model_analyzer
  ]
```

* Return to DAGs list page in Airflow
* Trigger tfx_example_pipeline_DAG
* Wait for pipeline to complete
  * All dark green
  * Use refresh on right side or refresh page

![Analyzing model performance](images/step6.png)

### Back on Jupyter:

* Open step6.ipynb
* Follow the notebook

![Analyzing model performance](images/step6notebook.png)

For a more extensive example of using TFMA to analyze model
performance, [see the examples on tensorflow.org](
https://www.tensorflow.org/tfx/model_analysis)

## Step 7: Deployment to production

If the new model is ready, make it so.

* If we’re replacing a model that is currently in production, first
make sure that the new one is better
* ModelValidator tells Pusher if the model is OK
* Pusher deploys SavedModels to well-known locations

Deployment targets receive new models from well-known locations

* TensorFlow Serving
* TensorFlow Lite
* TensorFlow JS
* TensorFlow Hub

### Components

ModelValidator

* Compares multiple versions of the trained model to make sure that the new version meets requirements

Pusher

* If the model passes ModelValidator, Pusher deploys the SavedModel to a well-known location

### In an editor:

```python
# Add the following code to ~/airflow/dags/tfx_example_pipeline.py
# Add appropriate imports
from tfx.components import ModelValidator
from tfx.components import Pusher

# Add components to the end of pipeline in create_pipeline()
  model_validator = ModelValidator(
      examples=example_gen.outputs.output,
      model=trainer.outputs.output)

  pusher = Pusher(
      model_export=trainer.outputs.output,
      model_blessing=model_validator.outputs.blessing,
      serving_model_dir=serving_model_dir)

# Add model_analyzer to return
  return [
      example_gen, statistics_gen, infer_schema, validate_stats, transform,
      trainer, model_analyzer, model_validator, pusher
  ]
```

* Return to DAGs list page in Airflow
* Trigger tfx_example_pipeline_DAG
* Wait for pipeline to complete
  * All dark green
  * Use refresh on right side or refresh page

![Deployment to production](images/step7.png)
